In [280]:
import pandas as pd

In [281]:
rationales = pd.read_excel('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] RATIONALES/IF-ELSE.xlsx', index_col=0)

In [282]:
rationales.rationales[15].lower().split('most likely answer: ')

['\n6. the question states that a child is in a meadow in spring. among the choices clean room, set table, play with toys, form opinions, gather flowers, the child would most likely gather flowers because meadows are filled with flowers and it is a common activity for children to pick flowers in a meadow during spring. therefore, the correct answer is (e) gather flowers. ']

In [332]:
rationales['split'] = rationales.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [333]:
rationales.split = rationales.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [334]:
import ast

# convert choices to list
rationales['choices'] = rationales.choices.apply(lambda x: ast.literal_eval(x))

ValueError: malformed node or string: ['park', 'coloring book', 'garden center', 'math problem', 'gravity']

In [335]:
# get the index of the correct answer
import numpy as np
rationales['correct_index'] = [{} for _ in range(len(rationales))]
for index, row in rationales.iterrows():
    for choice in row['choices']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)

In [336]:
# get the key of value min
rationales['LLM_answer'] =rationales.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [337]:
# HANDLE EXCEPTION
error = rationales[rationales.LLM_answer.isna()]
error.head(5)

,question,choices,answer,rationales,split,correct_index,LLM_answer
47,"A human is hitch hiking near Hoboken, where is...","[new jersey, homes, department store, near ita...",new jersey,8. The correct answer is (A) new jersey. The q...,.,{},NaN
110,A sedentary lifestyle of always sitting down i...,"[talk, relaxing, pee, gardening, watch tv]",watch tv,1. The correct answer is (E) watch tv. The que...,.,{},NaN
596,He took a toll road and ended up in a state he...,"[new jersey, new york, minnesota, eastern unit...",connecticut,7.\nThe correct answer is (B) new york.\nReaso...,.,{},NaN
2448,"The man tried to learn how to play the game, b...","[michigan, quit, improved performance, forget,...",forget,\n9.\nThe correct answer is (B) quit. The ques...,.,{},NaN
2705,The row houses featured nearby red phone booth...,"[england, living less expensively, city, town,...",england,6. The correct answer is (A) england. The ques...,.,{},NaN


In [338]:
# get the index of the correct answer
import numpy as np
error['correct_index'] = [{} for _ in range(len(error))]
for index, row in error.iterrows():
    for choice in row['choices']:
        if choice in row['rationales']:
            row['correct_index'][choice] = row['rationales'].lower().index(choice)

<ipython-input-338-a40908d18621>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['correct_index'] = [{} for _ in range(len(error))]


In [339]:
# get the key of value min
error['LLM_answer'] = error.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

<ipython-input-339-1f6358438110>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error['LLM_answer'] = error.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)


In [340]:
print(error[error.LLM_answer.isna()].question)

Series([], Name: question, dtype: object)


In [341]:
rationales.LLM_answer = rationales.LLM_answer.fillna(error.LLM_answer)

In [342]:
(sum(rationales.LLM_answer == rationales.answer)) / len(rationales)

0.6497901842729429

In [294]:
main = pd.read_csv('/home/huy/Desktop/HCMUS/total.csv', index_col=0)

In [295]:
rationales

,question,choices,answer,rationales,split,correct_index,LLM_answer
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",math problem,"1. Given the statement ""There are 10 apples on...","1. given the statement ""there are 10 apples on...",{'math problem': 131},math problem
1,"A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",bridge,2. The statement indicates that John is a bum ...,"is (a) bus depot. thus, the correct answer is...",{'bus depot': 8},bus depot
2,A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",sincere,3. The statement states that a bad person plac...,"is (b) upright. thus, the correct answer is (...",{'upright': 8},upright
3,"A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",minnesota,4. The statement mentions a bald eagle flying ...,"is (c) minnesota. thus, the correct answer is...",{'minnesota': 8},minnesota
4,A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",corvette,5. The statement states that if you need somet...,"is (d) destroyer, as destroyers are known for...",{'destroyer': 8},destroyer
...,...,...,...,...,...,...,...
10957,what does someone have that causes them commit...,"[problems, distress, fear, go to jail, killer]",problems,8. Given the question 'what does someone have ...,8. given the question 'what does someone have ...,"{'problems': 110, 'distress': 227, 'fear': 237...",problems
10958,what is printed with ink and distributed daily?,"[fountain pen, squid, newspaper, book, printer]",newspaper,9. Given the question 'what is printed with in...,9. given the question 'what is printed with in...,"{'fountain pen': 195, 'squid': 209, 'newspaper...",newspaper
10959,when communicating with my boss what should i do,"[misunderstandings, transfer of information, l...",transfer of information,10. Given the question 'when communicating wit...,10. given the question 'when communicating wit...,"{'misunderstandings': 238, 'transfer of inform...",transfer of information
10960,where is a good place to obtain new soap?,"[supermarket, washing, cabinet, own home, sink]",supermarket,Answer: (A) Supermarket\n\nReasoning: A good p...,answer: (a) supermarket\n\nreasoning: a good p...,"{'supermarket': 12, 'washing': 228, 'cabinet':...",supermarket


In [296]:
main['question'] = main.input.apply(lambda x: x.split('\n')[0])
a = rationales.set_index('question')
b = main.set_index('question')

In [297]:
sum(b.loc[a.index]['llm_label'].values == a.answer.values) / len(b)

0.7457580733442802

In [298]:
rationales.to_csv('/home/huy/Desktop/HCMUS/distilling-step-by-step/[API] RATIONALES/IF-ELSE - full.csv')

In [304]:
re_prompt = rationales[rationales.LLM_answer != rationales.answer]

In [312]:
prompt_template= '''
Questions: %s \n
Answers: (A) %s, (B) %s, (C) %s, (D) %s, (E) %s \
Given the question '%s', which among the choices %s, %s, %s, %s %s, is the correct answer? Explain your reasoning using conditional statements.\n
'''

In [313]:
re_prompt['prompt'] = re_prompt.apply(lambda x: prompt_template % (x.question, x.choices[0], x.choices[1], x.choices[2], x.choices[3], x.choices[4], x.question, x.choices[0], x.choices[1], x.choices[2], x.choices[3], x.choices[4]), axis=1)

<ipython-input-313-0252d33b1357>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  re_prompt['prompt'] = re_prompt.apply(lambda x: prompt_template % (x.question, x.choices[0], x.choices[1], x.choices[2], x.choices[3], x.choices[4], x.question, x.choices[0], x.choices[1], x.choices[2], x.choices[3], x.choices[4]), axis=1)


In [315]:
re_prompt.to_csv('re_prompt.csv')

In [317]:
re_prompt = pd.read_csv('re_prompt.csv', index_col=0)

In [330]:
rationales.loc[re_prompt.index, 'rationales'] = re_prompt.rationales

In [331]:
rationales

,question,choices,answer,rationales,split,correct_index,LLM_answer
0,"""There are 10 apples on an apple tree. Three ...","[park, coloring book, garden center, math prob...",math problem,"1. Given the statement ""There are 10 apples on...","1. given the statement ""there are 10 apples on...",{'math problem': 131},math problem
1,"A John is a bum. Much like the stereotype, he...","[bus depot, beach, train station, bridge, bridge]",bridge,The correct answer is (C) train station.\n\nCo...,"is (a) bus depot. thus, the correct answer is...",{'bus depot': 8},bus depot
2,A bad person places little value on being hone...,"[excellent, upright, premium, competent, sincere]",sincere,Answer: The correct answer is (E) sincere.\n\n...,"is (b) upright. thus, the correct answer is (...",{'upright': 8},upright
3,"A bald eagle flies over St. Paul, where is it?","[texas, thermal, minnesota, canada, photograph]",minnesota,4. The statement mentions a bald eagle flying ...,"is (c) minnesota. thus, the correct answer is...",{'minnesota': 8},minnesota
4,A battleship is a powerful vessel. If you nee...,"[yatch, corvette, aircraft carrier, destroyer,...",corvette,Answer: The correct answer is (D) destroyer.\n...,"is (d) destroyer, as destroyers are known for...",{'destroyer': 8},destroyer
...,...,...,...,...,...,...,...
10957,what does someone have that causes them commit...,"[problems, distress, fear, go to jail, killer]",problems,8. Given the question 'what does someone have ...,8. given the question 'what does someone have ...,"{'problems': 110, 'distress': 227, 'fear': 237...",problems
10958,what is printed with ink and distributed daily?,"[fountain pen, squid, newspaper, book, printer]",newspaper,9. Given the question 'what is printed with in...,9. given the question 'what is printed with in...,"{'fountain pen': 195, 'squid': 209, 'newspaper...",newspaper
10959,when communicating with my boss what should i do,"[misunderstandings, transfer of information, l...",transfer of information,10. Given the question 'when communicating wit...,10. given the question 'when communicating wit...,"{'misunderstandings': 238, 'transfer of inform...",transfer of information
10960,where is a good place to obtain new soap?,"[supermarket, washing, cabinet, own home, sink]",supermarket,Answer: (A) Supermarket\n\nReasoning: A good p...,answer: (a) supermarket\n\nreasoning: a good p...,"{'supermarket': 12, 'washing': 228, 'cabinet':...",supermarket


In [321]:
(sum(rationales.LLM_answer == rationales.answer)) / len(rationales)

0.6472359058565955